In [118]:
import csv
import openpyxl
import re
import types
import time
import datetime
#from types import NoneType
from datetime import datetime
from datetime import date
from datetime import timezone
from datetime import timedelta

In [130]:
##########################
# Read in a file
##########################

file_load = input("MPD number? XXXX ")
print(file_load)

#
# Construct a file name from user input
#

path_to_file = '/Users/jameshayes/'
today = date.today()
yy = today.strftime("%Y")

file_name = f'{path_to_file}{yy}_mpd_{file_load}.txt'
print(file_name)

MPD number? XXXX 1083
1083
/Users/jameshayes/2022_mpd_1083.txt


In [131]:
#read in the MPD
with open (file_name) as f:
    bx2 = f.readlines()
    str1 =''.join(bx2)
    print(str1)

Mesoscale Precipitation Discussion 1083
NWS Weather Prediction Center College Park MD
319 PM EDT Tue Oct 11 2022

Areas affected...Transverse into the northern Peninsular Ranges of
southern CA

Concerning...Heavy rainfall...Flash flooding possible

Valid 111916Z - 120100Z

SUMMARY...Localized flash flooding will be possible over sensitive
terrain located in the Transverse and northern Peninsular Ranges
of southern CA. Rainfall rates of 0.5 to 1.0 in/hr are expected
through 00Z with slow movement of cells but cannot rule out rates
just over 1 in/hr.

DISCUSSION...GOES West visible satellite imagery showed expanding
Cu/Cb along the Transverse/Peninsular Ranges of southern CA with a
mature thunderstorm in the San Jacinto Mountains containing
MRMS-derived rainfall rates near 1 in/hr as of 1845Z. The presence
of a closed upper level low centered just west of the Channel
Islands and a broader trough over the west-central U.S. has placed
a col in the 850-300 mb mean flow over the Mohave Deser

In [132]:
#############################################
# Get the valid times of the MPD
#############################################

ValidTime = re.search('Valid.*', str1)
if ValidTime:
    print(ValidTime.group(0))
    ValidValid = ValidTime.group(0)
        
ValidSplice = ValidValid[6:]
ValidStart = ValidSplice[2:6]
ValidEnd = ValidSplice[12:16]
ValidDay = ValidSplice[0:2]
ValidDay =int(ValidDay)

Valid 111916Z - 120100Z


In [133]:
#########################################
# Set up the discussion number and date
#########################################

from datetime import date, datetime

meso1 = bx2[0]
meso2 = meso1[-5:]
date1 = bx2[2]
    
timeSpace = date1[-30:]
dayMonth=date1[15:22]
dayMonth=dayMonth.strip()
print(dayMonth)
year = date1[-5:]
year1 = year[:4]
finalTime = timeSpace[6:] 
month = date1[15:19]
day = date1[19:22]
day1 = date1[19:22]
day1 = (day1.strip())

#
# set up logic to convert month str to int
#

month_name = month
month_name = month_name.strip()
print(month_name)
datetime_object = datetime.strptime(month_name, "%b")
month_number = datetime_object.month
month_number = str(month_number)
print(f'The month number has been calculated as: {month_number}')

# Set up datestring to convert to timestamp

startHour = ValidStart[0:2]
startMinute = ValidStart[2:4]
#print(startMinute.rstrip('\t'))
#startMinute = startMinute.rstrip().rstrip()
stopHour = ValidEnd[0:2]
stopMinute = ValidEnd[2:4]

Oct 11
Oct
The month number has been calculated as: 10


In [134]:
#############################################
# Get the flood type (possible, likely, none)
#############################################

x11 = re.search('Concerning.*', str1)
if x11:
        x12 = x11.group(0)
        pat = re.compile('possible|likely')
        x13 = re.search(pat, x12)
        if x13 is None:
           floodtype = "none"
           print('The flood type is', floodtype)
        
        if x13:
           print(x13.group(0))
           flood = x13.group(0)
           floodtype = flood
           print('The flood type is', floodtype)

possible
The flood type is possible


In [135]:
#############################################
# Get the Lat/Lon info
#############################################

LatLon = re.search('LAT...LON.*', str1, re.M|re.S)
if LatLon:
   print(LatLon.group(0))
   LatLon1 = LatLon.group(0) 

LAT...LON   34511830 34511792 34471726 34441671 34371653
            34241625 34111600 34051585 33951578 33861578
            33781583 33761591 33771596 33791603 33821610
            33861621 33911628 33901644 33861647 33801646
            33731642 33661635 33591628 33501620 33421619
            33331620 33281624 33261634 33261650 33241660
            33251674 33381695 33741709 34101715 34131744
            34191767 34181783 34211806 34361831


In [136]:
##############################################
# Get the WFOs affected        
##############################################

WFOwfo = re.search('ATTN...WFO.*', str1, re.M)
if WFOwfo:
   #print WFOwfo.group(0)
   WFOwfo1 = WFOwfo.group(0)
                     
WFOs = WFOwfo1[13:]
#print WFOs
MetWFO = WFOs[:-3]
#print MetWFO
FinalWFO = re.sub('\...',' ', MetWFO)
print("The final WFO list is ", FinalWFO)

The final WFO list is  LOX PSR SGX VEF


In [137]:
##############################################
# Extract and process the Lat/Lon information
##############################################

LatLon2 = LatLon1[10:]
interim = LatLon2.strip('\n')
interim2 = re.split(r'\s{1,}', interim)

latlat = [None] * len(interim2)
lonlon = [None] * len(interim2)

latlatX = [None] * len(interim2)
lonlonX = [None] * len(interim2)
lonlonFinal = [None] * len(interim2)

lat = [len(interim2)]
lon = [len(interim2)]  

i =  1
while i < len(interim2):
    latlat[i] = interim2[i][:4]
    lonlon[i] = interim2[i][4:]
    latlatX[i] = (float(latlat[i])/100)
    lonlonX[i] = (float(lonlon[i])/100)
    if lonlonX[i] < 30:
       lonlonFinal[i] = ((lonlonX[i] + 100)*-1)
    else:
       lonlonFinal[i] = ((lonlonX[i])*-1)
    
       print(i, latlatX[i], lonlonFinal[i])       
    i = i + 1

In [398]:
#################################################
# Write the CSV files for ArcMAP online and ArcPY
##################################################
#Construct the CSV file name

#file_nameCSV = f'{path_to_file}{yy}_mpd_{file_load}.csv'

#with open(file_nameCSV, 'w') as file:
#    writer = csv.writer(file)
#    writer.writerow(aa)

#bbb = len(interim2)
#print(bbb)
#
#    for i in range(1, bbb): #)[1:]:
#        print(i, latlatX[i], lonlonFinal[i])
#        writer.writerow((latlatX[i], lonlonFinal[i]))
#        writer.writerow((latlatX[1], lonlonFinal[1]))
       

14


In [141]:
#
# Is it a leap year?
#

year1 = int(year1)


if (year1 % 400 == 0):
    leap_year = True
elif (year1 % 100 == 0):
    leap_year = False
elif (year1 % 4 == 0):
    leap_year = True
else:
    leap_year = False



month_number = str(month_number)
print(f'The month_number has been calculated as: {month_number}')

if month_number in (1, 3, 5, 7, 8, 10, 12):
    month_length = 31
elif month_number == 2:
    if leap_year:
        month_length = 29
    else:
        month_length = 28
else:
    month_length = 30

print(month_length)
month_number = int(month_number);print(f'Starting month number: {month_number}')       
    
stopHour = int(stopHour) 
startHour = int(startHour)
print(f'The stop hour is {stopHour} and the start hour is {startHour}')

if stopHour < startHour:
    day_UTC += 1

print(f"This day:{ValidDay} this month:{month_name} This month number:{month_number} This month length:{month_length},{startHour}")       
if day == month_length and startHour < 3:  
    day =1 
    startDay = day
        
if day == 1:
    if month_number == 12:
        month_number = 1
        year1 += 1
    else:
        month_number += 1
        
monthDay = (f'{month_name} {ValidDay}')   
print(monthDay)            

#
# Create the date string to compute the datestamp
#

dateStringStart = (f'{month_number}/{ValidDay}/{year1} {startHour}:{startMinute}');print(dateStringStart)
dateStringEnd = (f'{month_number}/{ValidDay}/{year1} {stopHour}:{stopMinute}');print(dateStringEnd)

The month_number has been calculated as: 10
30
Starting month number: 10
The stop hour is 1 and the start hour is 19


NameError: name 'day_UTC' is not defined

In [128]:
#
# setup the time stamps
#

import time
import datetime
from datetime import datetime

print(dateStringStart)
objStart = time.strptime(dateStringStart, "%m/%d/%Y %H:%M")
objStop = time.strptime(dateStringEnd,"%m/%d/%Y %H:%M")

tStampStart = time.mktime(objStart);print(tStampStart)
tStampStop = time.mktime(objStop);print(tStampStop)

diffTime = ((tStampStop - tStampStart)/60);print(diffTime)
diffTime = int(diffTime)

10/5/2022 6:00
1664964000.0
1664982000.0
300.0


In [129]:
######x###########################################
# Write the other information to an Excel file
##################################################

from openpyxl import load_workbook
qb22 = int(file_load)
print(qb22)
qb33 = qb22 + 1
print(qb33)


file_path_xls = f'{path_to_file}MPD{yy}.xlsx'
print(file_path_xls)


wb = load_workbook(file_path_xls)
print(wb)
sheet = wb.active

# write out the header

print("Writing out the header to the Excel file...")

a1 = sheet['A1']
a1.value = "MPD#"
b1 = sheet['B1']
b1.value = "Date"
c1 = sheet['C1']
c1.value = "Cat"
d1 = sheet['D1']
d1.value = "Start"
e1 = sheet['E1']
e1.value = "Stop"
f1 = sheet['F1']
f1.value = "Time"
g1 = sheet['G1']
g1.value = "Area"
h1 = sheet['H1']
h1.value = "WFO"

# Write the data to the correct locations

print
print("Writing the MPD info to the file...")

# First the MPD#
aNext = sheet.cell(row = qb33, column = 1)
aNext.value = qb22

#Next the date
bNext = sheet.cell(row = qb33, column = 2)
bNext.value = monthDay

#Next the category of flooding
cNext = sheet.cell(row = qb33, column = 3)
cNext.value = floodtype

# Start time of the MPD
dNext = sheet.cell(row = qb33, column = 4)
dNext.value = ValidStart

# End of the MPD
eNext = sheet.cell(row = qb33, column = 5)
eNext.value = ValidEnd

# Get the time length of the MPD (under construction)
fNext = sheet.cell(row = qb33, column = 6)
fNext.value = diffTime

# Get the Area (under construction)
#gNext sheet.cell(row = qb33, column = 7)
#gNext.value = Area

# WFOs in the MPD
hNext = sheet.cell(row = qb33, column = 8)
hNext.value = FinalWFO


wb.save(file_path_xls)

1075
1076
/Users/jameshayes/MPD2022.xlsx
Writing out the header to the Excel file...
Writing the MPD info to the file...
